In [1]:
with open('data.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  421


In [3]:
print(text[:1000])

आत्मा सर्वत्र व्याप्यते। मनः चित्तसंग्रहाय कार्यं करोति। जीवनं दुःखसुखयोः मिश्रितं भवति। धर्मः आत्मनः प्रगतिपथः। अर्थस्य साधनं आवश्यकं, किंतु अहंकारं त्यजेत्। कामे च संयमः धर्मेण सह योजनीयः। मोक्षाय साधना अनिवार्या। सत्यं वद, अहिंसा चर, क्षमा धर्तु। मित्राणां संगः सुखदायकः, शत्रूणां संगः दुखदायकः। गुरुशिष्ययोः संबंधः ज्ञानस्य आधारः। 
विद्या विज्ञानयोः संयोजनं मनसः शान्त्यै उपयुक्तम्। संगीतं कला च हृदयशान्तये उपयोगी।




In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 ,ंःअआउकखगचजञणतथदधनपबभमयरलवशषसहािीुूृेैो्।
43


In [8]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("विद्या विज्ञानयोः"))
print(decode(encode("विद्या विज्ञानयोः")))

[27, 33, 17, 41, 24, 32, 1, 27, 33, 12, 41, 13, 32, 19, 24, 40, 4]
विद्या विज्ञानयोः


In [9]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([421]) torch.int64
tensor([ 6, 15, 41, 23, 32,  1, 30, 25, 41, 27, 15, 41, 25,  1, 27, 41, 24, 32,
        20, 41, 24, 15, 38, 42,  1, 23, 19,  4,  1, 11, 33, 15, 41, 15, 30,  3,
        10, 41, 25, 31, 32, 24,  1,  8, 32, 25, 41, 24,  3,  1,  8, 25, 40, 15,
        33, 42,  1, 12, 34, 27, 19,  3,  1, 17, 35,  4,  9, 30, 35,  9, 24, 40,
         4,  1, 23, 33, 28, 41, 25, 33, 15,  3,  1, 22, 27, 15, 33, 42,  1, 18,
        25, 41, 23,  4,  1,  6, 15, 41, 23, 19,  4,  1, 20, 41, 25, 10, 15, 33,
        20, 16,  4, 42,  1,  5, 25, 41, 16, 30, 41, 24,  1, 30, 32, 18, 19,  3,
         1,  6, 27, 28, 41, 24,  8,  3,  2,  1,  8, 33,  3, 15, 35,  1,  5, 31,
         3,  8, 32, 25,  3,  1, 15, 41, 24, 12, 38, 15, 41, 42,  1,  8, 32, 23,
        38,  1, 11,  1, 30,  3, 24, 23,  4,  1, 18, 25, 41, 23, 38, 14,  1, 30,
        31,  1, 24, 40, 12, 19, 34, 24,  4, 42,  1, 23, 40,  8, 41, 29, 32, 24,
         1, 30, 32, 18, 19, 32,  1,  5, 19, 33, 27, 32, 25, 41, 24, 32, 42,  1,
        30

In [10]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
train_data[:block_size+1]

tensor([ 6, 15, 41, 23, 32,  1, 30, 25, 41])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([6]) the target: 15
when input is tensor([ 6, 15]) the target: 41
when input is tensor([ 6, 15, 41]) the target: 23
when input is tensor([ 6, 15, 41, 23]) the target: 32
when input is tensor([ 6, 15, 41, 23, 32]) the target: 1
when input is tensor([ 6, 15, 41, 23, 32,  1]) the target: 30
when input is tensor([ 6, 15, 41, 23, 32,  1, 30]) the target: 25
when input is tensor([ 6, 15, 41, 23, 32,  1, 30, 25]) the target: 41


In [13]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [14]:

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[ 1,  5, 31, 33,  3, 30, 32,  1],
        [42,  1,  8, 32, 23, 38,  1, 11],
        [ 4,  2,  1, 28, 15, 41, 25, 36],
        [30, 32,  1, 11, 25,  2,  1,  8]])
targets:
torch.Size([4, 8])
tensor([[ 5, 31, 33,  3, 30, 32,  1, 11],
        [ 1,  8, 32, 23, 38,  1, 11,  1],
        [ 2,  1, 28, 15, 41, 25, 36, 14],
        [32,  1, 11, 25,  2,  1,  8, 41]])


In [15]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

when input is [1] the target: 5
when input is [1, 5] the target: 31
when input is [1, 5, 31] the target: 33
when input is [1, 5, 31, 33] the target: 3
when input is [1, 5, 31, 33, 3] the target: 30
when input is [1, 5, 31, 33, 3, 30] the target: 32
when input is [1, 5, 31, 33, 3, 30, 32] the target: 1
when input is [1, 5, 31, 33, 3, 30, 32, 1] the target: 11
when input is [42] the target: 1
when input is [42, 1] the target: 8
when input is [42, 1, 8] the target: 32
when input is [42, 1, 8, 32] the target: 23
when input is [42, 1, 8, 32, 23] the target: 38
when input is [42, 1, 8, 32, 23, 38] the target: 1
when input is [42, 1, 8, 32, 23, 38, 1] the target: 11
when input is [42, 1, 8, 32, 23, 38, 1, 11] the target: 1
when input is [4] the target: 2
when input is [4, 2] the target: 1
when input is [4, 2, 1] the target: 28
when input is [4, 2, 1, 28] the target: 15
when input is [4, 2, 1, 28, 15] the target: 41
when input is [4, 2, 1, 28, 15, 41] the target: 25
when input is [4, 2, 1, 28,

In [16]:
print(xb)

tensor([[ 1,  5, 31, 33,  3, 30, 32,  1],
        [42,  1,  8, 32, 23, 38,  1, 11],
        [ 4,  2,  1, 28, 15, 41, 25, 36],
        [30, 32,  1, 11, 25,  2,  1,  8]])


In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)